In [ ]:
from pedestrians_video_2_carla.data.carla.carla_recorded_dataset import CarlaRecordedDataset
from pedestrians_video_2_carla.data.carla.skeleton import CARLA_SKELETON

ds = CarlaRecordedDataset(
    '/outputs/CarlaRecordedDataModule/subsets/f9e757d00f25bfcba82d53ce5b204c99/train.hdf5',
    nodes=CARLA_SKELETON,
)

In [ ]:
len(ds)

In [ ]:
from pedestrians_video_2_carla.utils.tensors import get_bboxes
import torch

gt_projection_2d = torch.stack([ds[i][1]['projection_2d'] for i in range(len(ds))])

In [ ]:
gt_projection_2d.shape

In [ ]:
from pedestrians_video_2_carla.transforms.hips_neck import HipsNeckExtractor

hn_extractor = HipsNeckExtractor(input_nodes=CARLA_SKELETON)
hn_shift, hn_scale = hn_extractor.get_shift_scale(gt_projection_2d)

In [ ]:
from pedestrians_video_2_carla.transforms.bbox import BBoxExtractor

bb_extractor = BBoxExtractor(input_nodes=CARLA_SKELETON)
bb_shift, bb_scale = bb_extractor.get_shift_scale(gt_projection_2d)

In [ ]:
hn_bb_ratio = hn_scale / bb_scale
hn_bb_ratio_clips_avg = hn_bb_ratio.mean(dim=1)
hn_bb_ratio_avg = hn_bb_ratio_clips_avg.mean()

In [ ]:
hn_bb_ratio_avg

In [ ]:
bb_projection_2d = (gt_projection_2d - bb_shift[..., None, :]) / bb_scale[..., None, None]
hn_bb_shift, hn_bb_scale = hn_extractor.get_shift_scale(bb_projection_2d)

In [ ]:
hn_bb_scale_clips_avg = hn_bb_scale.mean(dim=1)
hn_bb_scale_avg = hn_bb_scale_clips_avg.mean()

In [ ]:
hn_bb_scale_avg

In [ ]:
hn_bb_shift_clips_avg = hn_bb_shift.mean(dim=1)
hn_bb_shift_avg = hn_bb_shift_clips_avg.mean()

In [ ]:
hn_bb_shift_avg

In [ ]:
bb_scale[0,0]

In [ ]:
hn_scale[0,0]

In [ ]:
bb_scale[0,0] * hn_bb_scale_avg

In [ ]:
%matplotlib widget

from matplotlib import pyplot as plt

diff_scale = (bb_scale * hn_bb_scale_avg - hn_scale).mean(dim=1)
gt_scale = hn_scale.mean(dim=1)

plt.plot(diff_scale.cpu().numpy())

In [ ]:
torch.allclose(bb_scale * hn_bb_scale_avg, hn_scale, atol=1e0, rtol=1e-1)

In [ ]:
%matplotlib widget

plt.plot((bb_shift[...,0] - hn_shift[...,0]).mean(dim=1).cpu().numpy())


In [ ]:
torch.allclose(bb_shift[...,0], hn_shift[...,0], atol=1e0, rtol=1e-1)

In [ ]:
torch.allclose((bb_shift + (bb_scale * hn_bb_shift_avg)[..., None])[...,1], hn_shift[...,1], atol=1e0, rtol=1e-1)